# Allstate Purchase Prediction

In this project, feature selection is performed before buliding multinomial logistic regression to predict the purchase likelyhood. Naive bayes is implemented from scatch to tackle the limitation of the original one can only work on binary categorical predictors.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as st
import sympy
import scipy

## Multinomail Logistic Regression

In [2]:
# All the below functions are written by my instructor Dr. Ming Long Lam
# Define a function to visualize the percent of a particular target category by a nominal predictor
def TargetPercentByNominal (
   targetVar,       # target variable
   predictor):      # nominal predictor

   countTable = pandas.crosstab(index = predictor, columns = targetVar, margins = True, dropna = True)
   x = countTable.drop('All', 1)
   percentTable = countTable.div(x.sum(1), axis='index')*100

   print("Frequency Table: \n")
   print(countTable)
   print( )
   print("Percent Table: \n")
   print(percentTable)
    
    
# A function that find alias parameters (redundant columns)
def find_alias(df, inds):
    alias = []
    columns = df.columns
    for i in range(len(df.columns)):
        if i not in inds:
            alias.append(columns[i])
    return alias


# A function that returns the columnwise product of two dataframes (must have same number of rows)
def create_interaction (df1, df2):
    name1 = df1.columns
    name2 = df2.columns
    result_df = pd.DataFrame()
    for col1 in name1:
        for col2 in name2:
            interaction_col = col1 + " * " + col2
            result_df[interaction_col] = df1[col1] * df2[col2]
    return result_df

# A function that find the non-aliased columns, fit a logistic model, and return the full parameter estimates
def build_mnlogit (fullX, y, debug = 'N'):
    # Number of all parameters
    nFullParam = fullX.shape[1]

    # Number of target categories
    y_category = y.cat.categories
    nYCat = len(y_category)

    # Find the non-redundant columns in the design matrix fullX
    reduced_form, inds = sympy.Matrix(fullX.values).rref()

    # These are the column numbers of the non-redundant columns
    if (debug == 'Y'):
        print('Column Numbers of the Non-redundant Columns:')
        print(inds)
        print()

    # Extract only the non-redundant columns for modeling
    X = fullX.iloc[:, list(inds)]

    # The number of free parameters
    thisDF = len(inds) * (nYCat - 1)

    # Build a multionomial logistic model
    logit = st.MNLogit(y, X)
    thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
    thisParameter = thisFit.params
    thisLLK = logit.loglike(thisParameter.values)

    if (debug == 'Y'):
        print(thisFit.summary())
        print("Model Parameter Estimates:\n", thisParameter)
        print("Model Log-Likelihood Value =", thisLLK)
        print("Number of Free Parameters =", thisDF)
        print()

    # Recreat the estimates of the full parameters
    workParams = pd.DataFrame(np.zeros(shape = (nFullParam, (nYCat - 1))))
    workParams = workParams.set_index(keys = fullX.columns)
    fullParams = pd.merge(workParams, thisParameter, how = "left", left_index = True, right_index = True)
    fullParams = fullParams.drop(columns = '0_x').fillna(0.0)

    # Return model statistics
    return (inds, thisLLK, thisDF, fullParams)


# Define a function that performs the Chi-square test
def ChiSquareTest (
    xCat,           # input categorical feature
    yCat,           # input categorical target variable
    debug = 'N'     # debugging flag (Y/N) 
    ):

    obsCount = pd.crosstab(index = xCat, columns = yCat, margins = False, dropna = True)
    cTotal = obsCount.sum(axis = 1)
    rTotal = obsCount.sum(axis = 0)
    nTotal = np.sum(rTotal)
    expCount = np.outer(cTotal, (rTotal / nTotal))

    if (debug == 'Y'):
        print('Observed Count:\n', obsCount)
        print('Column Total:\n', cTotal)
        print('Row Total:\n', rTotal)
        print('Overall Total:\n', nTotal)
        print('Expected Count:\n', expCount)
        print('\n')
       
    chiSqStat = ((obsCount - expCount)**2 / expCount).to_numpy().sum()
    chiSqDf = (obsCount.shape[0] - 1.0) * (obsCount.shape[1] - 1.0)
    chiSqSig = scipy.stats.chi2.sf(chiSqStat, chiSqDf)

    cramerV = chiSqStat / nTotal
    if (cTotal.size > rTotal.size):
        cramerV = cramerV / (rTotal.size - 1.0)
    else:
        cramerV = cramerV / (cTotal.size - 1.0)
    cramerV = np.sqrt(cramerV)

    return(chiSqStat, chiSqDf, chiSqSig, cramerV)

# Define a function that performs the Deviance test
def DevianceTest (
    xInt,           # input interval feature
    yCat,           # input categorical target variable
    debug = 'N'     # debugging flag (Y/N) 
    ):

    y = yCat.astype('category')

    # Model 0 is yCat = Intercept
    X = numpy.where(yCat.notnull(), 1, 0)
    objLogit = smodel.MNLogit(y, X)
    thisFit = objLogit.fit(method = 'newton', full_output = True, maxiter = 100, tol = 1e-8)
    thisParameter = thisFit.params
    LLK0 = objLogit.loglike(thisParameter.values)

    if (debug == 'Y'):
        print(thisFit.summary())
        print("Model Log-Likelihood Value =", LLK0)
        print('\n')

    # Model 1 is yCat = Intercept + xInt
    X = smodel.add_constant(xInt, prepend = True)
    objLogit = smodel.MNLogit(y, X)
    thisFit = objLogit.fit(method = 'newton', full_output = True, maxiter = 100, tol = 1e-8)
    thisParameter = thisFit.params
    LLK1 = objLogit.loglike(thisParameter.values)

    if (debug == 'Y'):
        print(thisFit.summary())
        print("Model Log-Likelihood Value =", LLK1)

    # Calculate the deviance
    devianceStat = 2.0 * (LLK1 - LLK0)
    devianceDf = (len(y.cat.categories) - 1.0)
    devianceSig = scipy.stats.chi2.sf(devianceStat, devianceDf)

    mcFaddenRSq = 1.0 - (LLK1 / LLK0)

    return(devianceStat, devianceDf, devianceSig, mcFaddenRSq)

In [13]:
# Loat the data set
purchases = pd.read_csv("Purchase_Likelihood.csv")

In [14]:
purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665249 entries, 0 to 665248
Data columns (total 4 columns):
group_size        665249 non-null int64
homeowner         665249 non-null int64
married_couple    665249 non-null int64
A                 665249 non-null int64
dtypes: int64(4)
memory usage: 20.3 MB


In [15]:
# Specify A as a categorical response
y = purchases['A'].astype('category')
purchases.drop(['A'], axis=1, inplace=True)

# Specify nominal features as categorical predictors
nom_features = ['group_size', 'homeowner', 'married_couple']
purchases = purchases[nom_features].astype('category')
purchases = pd.get_dummies(purchases)

group_size = purchases.iloc[:, 0:4]
homeowner = purchases.iloc[:, 4:6]
married_couple = purchases.iloc[:, 6:]
    
# Create columns for the group_size * homeowner interaction effect
group_size_homeowner = create_interaction(group_size, homeowner)

# Create columns for the homeowner * married_couple
homeowner_married_couple = create_interaction(homeowner, married_couple)

In [16]:
# Intercept only model
design_X = pd.DataFrame(y.where(y.isnull(), 1))
INDS0, LLK0, DF0, full_params0 = build_mnlogit(design_X, y)

# Intercept + group_size
design_X = st.add_constant(group_size, prepend=True)
INDS01, LLK_01, DF_01, full_params01 = build_mnlogit (design_X, y)
test_dev_01 = round(2 * (LLK_01 - LLK0), 4)
test_DF_01 = DF_01 - DF0
test_Pvalue_01 = scipy.stats.chi2.sf(test_dev_01, test_DF_01)
test_importance_01 = round(-np.log10(test_Pvalue_01), 4)
test_alias_01 = find_alias(design_X, INDS01)
print('Deviance Chi-Square Test for Model: ' + 'Intercept + group_size')
print('Chi-Square Statistic = ', test_dev_01)
print('  Degreee of Freedom = ', test_DF_01)
print('        Significance = ', test_Pvalue_01)
print('          Importance = ', test_importance_01)
print('  Aliased parameters = ', test_alias_01)
print()


Optimization terminated successfully.
         Current function value: 0.895013
         Iterations 5


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Optimization terminated successfully.
         Current function value: 0.894271
         Iterations 5
Deviance Chi-Square Test for Model: Intercept + group_size
Chi-Square Statistic =  987.5766
  Degreee of Freedom =  6
        Significance =  4.347871528385848e-210
          Importance =  209.3617
  Aliased parameters =  ['group_size_4']



In [18]:
# Intercept + group_size + homeowner
design_X = design_X.join(homeowner)
INDS012, LLK_012, DF_012, full_params012 = build_mnlogit (design_X, y)
test_dev_012 = round(2 * (LLK_012 - LLK_01), 4)
test_DF_012 = DF_012 - DF_01
test_Pvalue_012 = scipy.stats.chi2.sf(test_dev_012, test_DF_012)
test_importance_012 = round(-np.log10(test_Pvalue_012), 4)
test_alias_012 = find_alias(design_X, INDS012)
print('Deviance Chi-Square Test for Model: ' + 'Intercept + group_size + homeowner')
print('Chi-Square Statistic = ', test_dev_012)
print('  Degreee of Freedom = ', test_DF_012)
print('        Significance = ', test_Pvalue_012)
print('          Importance = ', test_importance_012)
print('  Aliased parameters = ', test_alias_012)
print()

# Intercept + group_size + homeowner + married_couple
design_X = design_X.join(married_couple)
INDS0123, LLK_0123, DF_0123, full_params0123 = build_mnlogit (design_X, y)
test_dev_0123 = round(2 * (LLK_0123 - LLK_012), 4)
test_DF_0123 = DF_0123 - DF_012
test_Pvalue_0123 = scipy.stats.chi2.sf(test_dev_0123, test_DF_0123)
test_importance_0123 = round(-np.log10(test_Pvalue_0123), 4)
test_alias_0123 = find_alias(design_X, INDS0123)
print('Deviance Chi-Square Test for Model: ' + 'Intercept + group_size + homeowner + married_couple')
print('Chi-Square Statistic = ', test_dev_0123)
print('  Degreee of Freedom = ', test_DF_0123)
print('        Significance = ', test_Pvalue_0123)
print('          Importance = ', test_importance_0123)
print('  Aliased parameters = ', test_alias_0123)
print()

# Intercept + group_size + homeowner + married_couple + group_size * homeowner
design_X = design_X.join(group_size_homeowner)
INDS01234, LLK_01234, DF_01234, full_params01234 = build_mnlogit (design_X, y)
test_dev_01234 = round(2 * (LLK_01234 - LLK_0123), 4)
test_DF_01234 = DF_01234 - DF_0123
test_Pvalue_01234 = scipy.stats.chi2.sf(test_dev_01234, test_DF_01234)
test_importance_01234 = round(-np.log10(test_Pvalue_01234), 4)
test_alias_01234 = find_alias(design_X, INDS01234)
print('Deviance Chi-Square Test for Model: ' + 'Intercept + group_size + homeowner + married_couple + group_size * homeowner')
print('Chi-Square Statistic = ', test_dev_01234)
print('  Degreee of Freedom = ', test_DF_01234)
print('        Significance = ', test_Pvalue_01234)
print('          Importance = ', test_importance_01234)
print('  Aliased parameters = ', test_alias_01234)
print()

# Intercept + group_size + homeowner + married_couple + group_size * homeowner + homeowner * married_couple
design_X = design_X.join(homeowner_married_couple)

# Number of all parameters
n_full_param = design_X.shape[1]

# Number of target categories
y_category = y.cat.categories
n_y_cat = len(y_category)

# Find the non-redundant columns in the design matrix fullX
reduced_form_012345, INDS012345 = sympy.Matrix(design_X.values).rref()

# These are the column numbers of the non-redundant columns
print('Column Numbers of the Non-redundant Columns:')
print(INDS012345)

# Extract only the non-redundant columns for modeling
X = design_X.iloc[:, list(INDS012345)]

# The number of free parameters
DF_012345 = len(INDS012345) * (n_y_cat - 1)

# Build a multionomial logistic model
logit = st.MNLogit(y, X)
fit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
fit_params = fit.params
LLK_012345 = logit.loglike(fit_params.values)

print(fit.summary())
print("Model Parameter Estimates:\n", fit_params)
print("Model Log-Likelihood Value =", LLK_012345)
print("Number of Free Parameters =", DF_012345)
print()

# Recreat the estimates of the full parameters
work_params = pd.DataFrame(np.zeros(shape = (n_full_param, (n_y_cat - 1))))
work_params = work_params.set_index(keys = design_X.columns)
full_params = pd.merge(work_params, fit_params, how = "left", left_index = True, right_index = True)
full_params = full_params.drop(columns = '0_x').fillna(0.0)

# Calculate the statistics of the full model
test_dev_012345 = round(2 * (LLK_012345 - LLK_01234), 4)
test_DF_012345 = DF_012345 - DF_01234
test_Pvalue_012345 = scipy.stats.chi2.sf(test_dev_012345, test_DF_012345)
test_importance_012345 = round(-np.log10(test_Pvalue_012345), 4)
test_alias_012345 = find_alias(design_X, INDS012345)
print('Deviance Chi-Square Test for Model: ' +'Intercept + group_size + homeowner + married_couple + group_size * homeowner + homeowner * married_couple')
print('Chi-Square Statistic = ', test_dev_012345)
print('  Degreee of Freedom = ', test_DF_012345)
print('        Significance = ', test_Pvalue_012345)
print('          Importance = ', test_importance_012345)
print('  Aliased parameters = ', test_alias_012345)
print()

'''
For each of the sixteen possible value combinations of the three features, 
calculate the predicted probabilities for A = 0, 1, 2 based on the multinomial logistic model.  
'''
# Sixteen combination of group_size, homeowner & married_couple
df_ghm = pd.DataFrame({'group_size': [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
                       'homeowner': [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
                       'married_couple': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]})
print("Sixteen possible value combinations of the three features:")    
print(df_ghm)
print()

# Specify nominal features as categorical predictors
ghm_features = ['group_size', 'homeowner', 'married_couple']
df_ghm = df_ghm[ghm_features].astype('category')
df_ghm = pd.get_dummies(df_ghm)

group_size_ghm = df_ghm.iloc[:, 0:4]
homeowner_ghm = df_ghm.iloc[:, 4:6]
married_couple_ghm = df_ghm.iloc[:, 6:]
    
# Create columns for the group_size * homeowner interaction effect
group_size_homeowner_ghm = create_interaction(group_size_ghm, homeowner_ghm)

# Create columns for the homeowner * married_couple
homeowner_married_couple_ghm = create_interaction(homeowner_ghm, married_couple_ghm)

design_X_ghm = df_ghm.join(group_size_homeowner_ghm)
design_X_ghm = design_X_ghm.join(homeowner_married_couple_ghm)
design_X_ghm = st.add_constant(design_X_ghm, prepend=True)

# Extract only the non-redundant columns for modeling
X_ghm = design_X_ghm.iloc[:, list(INDS012345)]

# Predict the sixteen combinations of the three features
pred_proba = fit.predict(X_ghm)

print('The predicted probabilities for A = 0, 1, 2 based on the multinomial logistic model.')
print(pred_proba)
print()

'''
What values of group_size, homeowner, and married_couple 
will maximize the odds value Prob(A=1) / Prob(A = 0)?  
What is that maximum odd value?
'''
df_combination = pd.DataFrame({'group_size': [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
                               'homeowner': [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
                               'married_couple': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]})
odds_1_0 = list(pred_proba.iloc[:, 1] / pred_proba.iloc[:, 0])
max_odds_1_0 = max(odds_1_0)
max_odds_1_0_idx = odds_1_0.index(max_odds_1_0)
print('Values of group_size, homeowner and married_couple that maximaize the odss value Prob(A=1)/Prob(A=0):')
print(df_combination.iloc[max_odds_1_0_idx, :].values)
print('The maximum odd value:', np.exp(fit_params.iloc[0,0] + fit_params.iloc[1,0]))
print()

'''
The odds ratio for group_size = 3 versus group_size = 1,and A = 2 versus A = 0.  Mathematically, 
the odds ratio is (Prob(A=2)/Prob(A=0) | group_size = 3) / ((Prob(A=2)/Prob(A=0) | group_size = 1).
'''
group_size_3 = fit_params.iloc[3, 1]
group_size_1 = fit_params.iloc[1, 1]
odds_ratio_A2_A0_g3_g1 = np.exp(group_size_3 - group_size_1)
print('The odds ratio for group_size = 3 versus group_size = 1, and A = 2 versus A = 0:', odds_ratio_A2_A0_g3_g1)
print()

'''
The odds ratio for homeowner = 1 versus homeowner = 0, and A = 0 versus A = 1? 
Mathematically, the odds ratio is (Prob(A=0)/Prob(A=1) | homeowner = 1) / ((Prob(A=0)/Prob(A=1) | homeowner = 0).
'''
odds_ratio_A0_A1_h1_h0 = np.exp(fit_params.iloc[4,0])
print('The odds ratio for homeowner = 1 versus homeowner = 0, and A = 0 versus A = 1:', odds_ratio_A0_A1_h1_h0)

Optimization terminated successfully.
         Current function value: 0.889861
         Iterations 5


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log10
  import sys


Deviance Chi-Square Test for Model: Intercept + group_size + homeowner
Chi-Square Statistic =  5867.7815
  Degreee of Freedom =  2
        Significance =  0.0
          Importance =  inf
  Aliased parameters =  ['group_size_4', 'homeowner_1']

Optimization terminated successfully.
         Current function value: 0.889797
         Iterations 5
Deviance Chi-Square Test for Model: Intercept + group_size + homeowner + married_couple
Chi-Square Statistic =  84.578
  Degreee of Freedom =  2
        Significance =  4.3064623511902606e-19
          Importance =  18.3659
  Aliased parameters =  ['group_size_4', 'homeowner_1', 'married_couple_1']

Optimization terminated successfully.
         Current function value: 0.889606
         Iterations 5
Deviance Chi-Square Test for Model: Intercept + group_size + homeowner + married_couple + group_size * homeowner
Chi-Square Statistic =  254.0781
  Degreee of Freedom =  6
        Significance =  5.512174780169455e-52
          Importance =  51.2587
 

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is dep

## Implement Naive Bayes

In [19]:
# All the below functions are written by my instructor Dr. Ming Long Lam
# Define a function to visualize the percent of a particular target category by a nominal predictor
def RowWithColumn (
   rowVar,          # Row variable
   columnVar,       # Column predictor
   show = 'ROW'):   # Show ROW fraction, COLUMN fraction, or BOTH table

   countTable = pd.crosstab(index = rowVar, columns = columnVar, margins = False, dropna = True)
   countTable['Total'] = countTable.sum(1)
   print("Frequency Table: \n", countTable)
   print( )
   
   rowFraction = None
   if (show == 'ROW'):
       rowFraction = countTable.div(countTable['Total'], axis='index')
       print("Row Fraction Table: \n", rowFraction)
       print( )
       
   return countTable, rowFraction


# Define a function that calculate the Cramer's V
def cramers_v (
    x_cat,           # input categorical feature
    y_cat,           # input categorical target variable
    debug = 'N'     # debugging flag (Y/N) 
    ):

    obs_count = pd.crosstab(index = x_cat, columns = y_cat, margins = False, dropna = True)
    col_tot = obs_count.sum(axis = 1)
    row_tot = obs_count.sum(axis = 0)
    tot = np.sum(row_tot)
    exp_count = np.outer(col_tot, (row_tot / tot))

    if (debug == 'Y'):
        print('Observed Count:\n', obs_count)
        print('Column Total:\n', col_tot)
        print('Row Total:\n', row_tot)
        print('Overall Total:\n', tot)
        print('Expected Count:\n', exp_count)
        print('\n')
       
    chi_sq_stat = ((obs_count - exp_count)**2 / exp_count).to_numpy().sum()
    
    cramerV = chi_sq_stat / tot
    if (col_tot.size > row_tot.size):
        cramerV = cramerV / (row_tot.size - 1.0)
    else:
        cramerV = cramerV / (col_tot.size - 1.0)
    cramerV = np.sqrt(cramerV)

    return cramerV

In [20]:
# Loat the dataset
purchases = pd.read_csv("Purchase_Likelihood.csv")  

In [21]:
'''
The frequency counts and the Class Probabilities of the target variable.
'''
count_table_A = pd.crosstab(index=purchases['A'], columns=['Count'], margins=True)
count_table_A.drop(['All'], axis=1, inplace=True)
count_table_A.rename(index={'All': 'Total'}, inplace=True)
count_table_A['Class Probability'] = count_table_A.div(count_table_A.loc['Total', ], axis='columns')
print('The frequency counts and the Class Probabilities of the target variable:')
print(count_table_A)
print()

'''
The crosstabulation table of the target variable by the feature group_size.  
The table contains the frequency counts.
'''
print('The crosstabulation table of the target variable by the feature group_size:')
count_table_group, count_table_group_fraction = RowWithColumn(purchases['A'], purchases['group_size'])

'''
The crosstabulation table of the target variable by the feature homeowner.  
The table contains the frequency counts.
'''
print('The crosstabulation table of the target variable by the feature homeowner:')
count_table_homeowner, count_table_homeowner_fraction = RowWithColumn(purchases['A'], purchases['homeowner'])

'''
Tthe crosstabulation table of the target variable by the feature married_couple.  
The table contains the frequency counts.
'''
print('The crosstabulation table of the target variable by the feature married_couple:')
count_table_married_couple, count_table_married_couple_fraction = RowWithColumn(purchases['A'], purchases['married_couple'])

'''
Calculate the Cramer’s V statistics for the above three crosstabulations tables.  
Based on these Cramer’s V statistics, show which feature has the largest association with the target A.
'''
cat_pred = ['group_size', 'homeowner', 'married_couple']
cramerv_table = pd.DataFrame(index = cat_pred, columns = ['Test', "Cramer's V"])
for pred in cat_pred:
    cramerV = cramers_v(purchases[pred], purchases['A'])
    cramerv_table.loc[pred] = ['Chi-square', cramerV]
cramerv_table = cramerv_table.sort_values("Cramer's V", axis = 0, ascending = False)
print("Cramer’s V statistics:")
print(cramerv_table)
print()

'''
For each of the sixteen possible value combinations of the three features, 
calculate the predicted probabilities for A = 0, 1, 2 based on the Naïve Bayes model. 
'''
df_proba = pd.DataFrame({'group_size': [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
                         'homeowner': [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
                         'married_couple': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]})
df_proba['Prob(A = 0)'] = None
df_proba['Prob(A = 1)'] = None
df_proba['Prob(A = 2)'] = None

# A function that calculates the probability of the categories of a target variable
def naive_bayes(df_a, df_g, df_h, df_m, g, h, m):
    # Array that stores the probilities given A = 0 or A = 1 or A = 2
    p_arr = np.array([None, None, None])
    for i in range(3):
        p = df_a.iloc[i, 1] * df_g.iloc[i, g] * df_h.iloc[i, h] * df_m.iloc[i, m]
        p_arr[i] = p
    p_arr = p_arr/np.sum(p_arr)
    return p_arr
    

# Loop through the rows of the dataframe
for i in range(df_proba.shape[0]):
    g = df_proba.iloc[i, 0] - 1 # Retrieve the group category
    h = df_proba.iloc[i, 1]     # Retrieve the homeowner category
    m = df_proba.iloc[i, 2]     # Retrieve the married_couple category
    
    p_arr = naive_bayes(count_table_A, count_table_group_fraction, count_table_homeowner_fraction, count_table_married_couple_fraction, g, h, m)
    
    df_proba.iloc[i, 3] = p_arr[0]    # Assign the probability of A = 0
    df_proba.iloc[i, 4] = p_arr[1]    # Assign the probability of A = 1
    df_proba.iloc[i, 5] = p_arr[2]    # Assign the probability of A = 2

print('The predicted probabilities for A = 0, 1, 2 based on the Naïve Bayes model:')
print(df_proba)
print()
          

'''          
Based on your model, calculate what values of group_size, homeowner, 
and married_couple will maximize the odds value Prob(A=1) / Prob(A = 0) and Whatthat maximum odd value is.         
'''         
odds = list(df_proba.iloc[:, 4]/df_proba.iloc[:, 3])
max_odds = max(odds)
max_odds_idx = odds.index(max_odds)
print('Values of group_size, homeowner, and married_couple will maximize the odds value Prob(A=1) / Prob(A = 0)')
print(df_proba.iloc[max_odds_idx, 0:3].values)
print('Maximum odd value:', max_odds)


from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
X = purchases.iloc[:, 0:3].astype('category')
y = purchases.iloc[:, 3].astype('category')
mnb.fit(X, y)

test= pd.DataFrame({'group_size': [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
                         'homeowner': [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
                         'married_couple': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]})
pred_proba = mnb.predict_proba(test)

The frequency counts and the Class Probabilities of the target variable:
col_0   Count  Class Probability
A                               
0      143691           0.215996
1      426067           0.640462
2       95491           0.143542
Total  665249           1.000000

The crosstabulation table of the target variable by the feature group_size:
Frequency Table: 
 group_size       1      2     3    4   Total
A                                           
0           115460  25728  2282  221  143691
1           329552  91065  5069  381  426067
2            74293  19600  1505   93   95491

Row Fraction Table: 
 group_size         1         2         3         4  Total
A                                                        
0           0.803530  0.179051  0.015881  0.001538    1.0
1           0.773475  0.213734  0.011897  0.000894    1.0
2           0.778010  0.205255  0.015761  0.000974    1.0

The crosstabulation table of the target variable by the feature homeowner:
Frequency Table: 
 